In [15]:
#Imports
import os
import pandas as pd
import numpy as np

In [16]:
# os.chdir("Desktop")

In [17]:
df1 = pd.read_csv("fake_news_train.csv")
df2= pd.read_csv("fake_news_test.csv")

In [18]:
#Lets set up the y variable first

y= df1["label"]

In [19]:
#let us evaluate the target variable for class imbalance
y.value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [20]:
#The classes are balanced- no need to apply oversampling or undersampling techniques

In [21]:
X= df1[["text"]]
text = X["text"]

In [22]:
X["text"]= X["text"].apply(lambda x:str(x))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
import re
import nltk
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

In [24]:
normalize_corpus = np.vectorize(normalize_document)

In [25]:
norm_corpus = normalize_corpus(text)

In [26]:
#Bag of words model

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()

In [27]:
from sklearn.model_selection import train_test_split as tts
X_train_cv, X_test_cv, y_train, y_test= tts(norm_corpus, y, test_size=0.3, random_state= 42)

In [28]:
X_train_cv = cv.fit_transform(X_train_cv)
X_test_cv= cv.transform(X_test_cv)

In [29]:
# Get all unique words in the corpus
vocab = cv.get_feature_names()

In [31]:
# Show document feature vectors
matrix = pd.DataFrame(X_train_cv.toarray(), columns=vocab)

In [32]:
matrix.head()

,00,000,0000,0001,000k,000s,001,002,0023z,004,...,进行过审查,都市如此,集体安全条约组织,集体安全条约组织与联合国专门机构,集体安全条约组织是多层面结构,集体安全条约组织正在积极促进帮助阿富汗进行冲突后重建以及消除来自该国的毒品威胁的国际努力,集体安全条约组织秘书长博尔久扎,集体提出创新办法,非法贩运毒品以及确保国际信息安全,预防和解决冲突
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
#We will be using the Naive Bayes algorithm on this dataset

In [34]:
from sklearn.naive_bayes import MultinomialNB

In [35]:
mnb= MultinomialNB()

In [36]:
mnb.fit(X_train_cv, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [37]:
y_pred= mnb.predict(X_test_cv)

In [38]:
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

In [39]:
accuracy_score(y_test, y_pred)

0.9028846153846154

In [40]:
confusion_matrix(y_test,y_pred)

array([[3038,  110],
       [ 496, 2596]])

In [41]:
roc_auc_score(y_test, y_pred)

0.9023216038109578

In [42]:
#Now let us see if we can improve our model by using N-grams. We will start with bigrams 

In [43]:
bv = CountVectorizer(ngram_range=(2,2))

In [44]:
X_train_bv, X_test_bv, y_train, y_test= tts(norm_corpus, y, test_size=0.3, random_state= 42)

In [45]:
X_train_bv = bv.fit_transform(X_train_bv)
X_test_bv= bv.transform(X_test_bv)

In [46]:
vocab = bv.get_feature_names()

In [47]:
matrix = pd.DataFrame(X_train_bv.toarray(), columns=vocab)

In [48]:
matrix.head()

,00 00,00 000,00 120,00 15,00 25,00 44,00 45,00 48,00 54,00 55,...,进行过审查 众所周知,都市如此 而且也是其国际公认的合法性的体现,集体安全条约组织 上海合作组织和独立国家联合体,集体安全条约组织与联合国专门机构 包括安全理事会反恐怖主义委员会和联合国毒品和犯罪问题办公室,集体安全条约组织是多层面结构 能对其成员国可能面临的当代广泛的挑战和威胁作出坚定回应,集体安全条约组织正在积极促进帮助阿富汗进行冲突后重建以及消除来自该国的毒品威胁的国际努力 该组织与联合国的关系在许多领域都得到成功的发展,集体安全条约组织秘书长博尔久扎 上海合作组织秘书长阿利莫夫以及独联体执行委员会副主席伊凡诺夫将发表讲话,集体提出创新办法 嗣后可推广到安全理事会重点关注的世界其他地区,非法贩运毒品以及确保国际信息安全 在此范畴内,预防和解决冲突 反恐
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
mnb.fit(X_train_bv, y_train)
y_pred_bigram= mnb.predict(X_test_bv)

In [50]:
accuracy_score(y_test, y_pred_bigram)

0.9238782051282052

In [51]:
confusion_matrix(y_test, y_pred_bigram)

array([[3117,   31],
       [ 444, 2648]])

In [52]:
roc_auc_score(y_test, y_pred_bigram)

0.9232780500073148

In [53]:
#Now let us see the results using a different model- TFIDF 

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [55]:
X_train_tf, X_test_tf, y_train, y_test= tts(norm_corpus, y, test_size=0.3, random_state= 42)

In [56]:
X_train_tf = tfidf.fit_transform(X_train_tf)
X_test_tf= tfidf.transform(X_test_tf)

In [57]:
#lets apply the Naive Bayes algorithm to this now

mnb.fit(X_train_tf, y_train)
y_pred_tfidf= mnb.predict(X_test_tf)

In [58]:
accuracy_score(y_test, y_pred_tfidf)

0.8426282051282051

In [59]:
confusion_matrix(y_test, y_pred_tfidf)

array([[3118,   30],
       [ 952, 2140]])

In [60]:
roc_auc_score(y_test, y_pred_tfidf)

0.8412894036501953

In [61]:
#it seems that our Bag of word bigram model provides the highest accuracy
#We will now use this model to predict using our real test set

In [62]:
#Revectorize our training and test dataset
df1["text"]= df1["text"].apply(lambda x:str(x))
df2["text"]= df2["text"].apply(lambda x:str(x))

In [63]:
train = np.array(df1.text)

In [64]:
test= np.array(df2.text)

In [65]:
text = np.concatenate((train, test), axis=0)

In [66]:
#Apply the normalize corpus function on the combined text array

X = normalize_corpus(text)

In [67]:
#Split into train and test once the overall vectorization is done

X_train = X[:20800]
X_test = X[20800:]

In [68]:
#Apply our bigram count vectorizer model on the train and test sets
X_train = bv.fit_transform(X_train)

In [69]:
X_test = bv.transform(X_test)

In [70]:
#Applying the naive bayes algorithm to get our final results

mnb.fit(X_train, y)
y_pred_test = mnb.predict(X_test)

In [71]:
#Our final results array
y_pred_test

array([0, 1, 1, ..., 0, 1, 0])